In [1]:
from wdtools import *

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
files = list_files(os.path.join(wdpath, 'Set001'))

In [3]:
files = [file for file in files if '~$' not in file]

In [4]:
set_ID = 'Set001'

In [5]:
start1 = time.time()
frames = []
for year in range(2017, 2023):
    #start2 = time.time()
    tx_dt = read_taxlot(year)
    #end2 = time.time()
    #print("it took {0} minutes to read taxlot in {1}".format(str(round((end2 - start2)/60, 0)), str(year)))
    for file in files:
        #print("processing {0} in {1}".format(file, str(year)))
        wd_dt = read_wd_table(setID = set_ID, file = file)  
        df = wd_dt[wd_dt.IDyear == str(year)].merge(tx_dt[['ORTaxlot', 'geometry']], 
                                                    on='ORTaxlot', 
                                                    how='left')
        frames.append(df)
        #print("processed {0} in {1}".format(file, str(year)))
df = pd.concat(frames, ignore_index=True)      
end1 = time.time()
print("it took {0} minutes to complete {1}".format(str(round((end1 - start1)/60, 0)), set_ID))

it took 16.0 minutes to complete Set001


In [6]:
df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'lots', 'lot', 'cnt_code', 'ORTaxlot', 'year', 'IDyear', 'geometry'],
      dtype='object')

In [7]:
df.reissuance_response_date.dtype

dtype('float64')

In [9]:
df['response_date'] = df['response_date'].dt.strftime("%Y-%m-%d")

In [10]:
df['received_date'] = df['received_date'].dt.strftime("%Y-%m-%d")

In [13]:
df['lots'] = df['lots'].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))

In [22]:
df.tail(3)

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,project_id,site_id,record_ID,lots,lot,cnt_code,ORTaxlot,year,IDyear,geometry
2493,WD2022-0301,06S01W32,"500,600,601,700,900",Hazelgreen Rd near Torvend Rd over the Pudding...,Silverton,Marion,Hazelgreen Rd Bridge Replacement,WD2022-0301 delin,45.00945,-122.84316,...,84297.0,139465,268,500 600 601 700 900,601,24,2406.00S01.00W3200--000000601,2022,2022,"MULTIPOLYGON (((706600.131 1196247.539, 706585..."
2494,WD2022-0301,06S01W32,"500,600,601,700,900",Hazelgreen Rd near Torvend Rd over the Pudding...,Silverton,Marion,Hazelgreen Rd Bridge Replacement,WD2022-0301 delin,45.00945,-122.84316,...,84297.0,139465,268,500 600 601 700 900,700,24,2406.00S01.00W3200--000000700,2022,2022,"MULTIPOLYGON (((706341.268 1196877.808, 706285..."
2495,WD2022-0301,06S01W32,"500,600,601,700,900",Hazelgreen Rd near Torvend Rd over the Pudding...,Silverton,Marion,Hazelgreen Rd Bridge Replacement,WD2022-0301 delin,45.00945,-122.84316,...,84297.0,139465,268,500 600 601 700 900,900,24,2406.00S01.00W3200--000000900,2022,2022,"MULTIPOLYGON (((711115.773 1196891.122, 711110..."


In [14]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:2992", geometry='geometry')

In [17]:
gdf.rename(columns={'wetdet_delin_number': 'wdID', 
                      'address_location_desc':'loc_desc', 
                      'Coord-Source': 'coord_src',
                      'DocumentName':'doc_name',
                      'DecisionLink':'doc_link',
                      'is_batch_file':'isbatfile',
                      'status_name': 'status_nm',
                      'received_date':'receiveddt', 
                      'response_date':'responsedt',
                      'reissuance_response_date':'reissuance', 
                      }, inplace=True)

In [5]:
gdf = gpd.read_file(os.path.join(inpath + '\\test', 'matched_records_with_lotID'))

In [6]:
# read all set001 data without merging
wd_df = combine_wd_table(setID = set_ID)

In [31]:
selectedID = wd_df.parcel_id.astype(str) != 'nan'

In [32]:
wd_df.loc[selectedID, 'notes'] = wd_df[selectedID]['parcel_id'].apply(lambda x: make_notes(x))

In [7]:
wd_df.record_ID.values

array([   1,    2,    3, ..., 1418, 1419, 1420], dtype=int64)

In [9]:
wd_df.shape

(1420, 21)

In [9]:
gdf.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'coord_src', 'doc_name',
       'doc_link', 'isbatfile', 'status_nm', 'receiveddt', 'responsedt',
       'reissuance', 'project_id', 'site_id', 'record_ID', 'lots', 'lot',
       'cnt_code', 'ORTaxlot', 'year', 'IDyear', 'geometry'],
      dtype='object')

In [10]:
counties = wd_df.county.unique()

In [11]:
count_records = []
for cnty in counties:
    count = len(wd_df[wd_df.county == cnty])
    #print(f'County {cnty} has {count} records')
    count_records.append(count)

In [12]:
count_records

[181, 148, 241, 21, 383, 109, 69, 268]

In [13]:
record_df = pd.DataFrame({'county': counties, 'rcount':count_records})

In [14]:
record_df['cum_count'] = record_df[['rcount']].cumsum(axis = 0, skipna = True).rcount.values

In [15]:
record_df

,county,rcount,cum_count
0,Clatsop,181,181
1,Coos,148,329
2,Deschutes,241,570
3,Harney,21,591
4,Jackson,383,974
5,Josephine,109,1083
6,Lake,69,1152
7,Marion,268,1420


In [16]:
record_dict = dict(zip(record_df.county[1:8], record_df.cum_count[0:7]))

In [17]:
record_dict

{'Coos': 181,
 'Deschutes': 329,
 'Harney': 570,
 'Jackson': 591,
 'Josephine': 974,
 'Lake': 1083,
 'Marion': 1152}

In [20]:
selected_cnty = gdf.county.isin(counties[1:])

In [21]:
gdf.loc[selected_cnty, 'updated_rID'] = gdf.loc[selected_cnty, 'record_ID'] + gdf[selected_cnty].county.map(record_dict)

In [22]:
gdf.loc[gdf.county == 'Clatsop', 'updated_rID'] = gdf.loc[gdf.county == 'Clatsop', 'record_ID']

In [24]:
gdf.loc[:, 'updated_rID'] = gdf.updated_rID.astype('int64', copy=False)

In [27]:
matched_rID = check_duplicates(gdf.updated_rID.values)[0]

In [28]:
unmatched_wd_df = wd_df[~wd_df.record_ID.isin(matched_rID)]

In [26]:
wd_df.shape

(1420, 21)

In [29]:
unmatched_wd_df.shape

(945, 21)

In [33]:
len(gdf.updated_rID.unique())/wd_df.shape[0]

0.9570422535211267

In [36]:
check_duplicates([tx for tx in set1gdf.ORTAXLOT.values if tx not in gdf.ORTaxlot.values])

([None], 1)

In [23]:
check_duplicates(unmatched_wd_df.parcel_id.values)[0]

['500 (partial)',
 '2500, ROWs (both partial)',
 '302 (partial)',
 '2100 (partial)',
 '204',
 '4600 (partial)',
 '10700',
 'NW 11th St ROW (partial)',
 '3602',
 '12308',
 '12400',
 '600 (partial)',
 '2310, ROW (both partial)',
 '2903',
 '8300',
 '1529',
 '408 (portion)',
 '1204 (partial)',
 '702',
 '5800 (partial)',
 '1500 (portion)',
 '2301',
 '923',
 '3400, ROW partial',
 '600, ROW partial',
 '1002 partial',
 '500 (portion)',
 '4302, ROW all partial',
 '5800 portion',
 '5800 partial',
 '2100, ROW partial',
 '3407',
 '2204',
 '903',
 '1409',
 '3800,ROW',
 '10890, ROW partial',
 '502',
 '2408',
 '18700',
 '8705',
 '1505',
 '2401, ROW',
 '802',
 '10200',
 '6500',
 '11100',
 '613',
 '620',
 '601p',
 '140p',
 '5000p',
 '2501',
 '3001',
 '901',
 '5000,ROW (all part)',
 '1420',
 '5001',
 '4000',
 '402p',
 '8400',
 '505',
 '300 portion',
 '700 portion',
 '99 partial',
 '5700',
 '406',
 '104 partial',
 '2101',
 '1403',
 '109 partial',
 '705 partial',
 '106 (portion)',
 '1701',
 '4400,WATER,Al

In [20]:
#gdf.to_file(os.path.join(inpath + '\\test', 'matched_records_with_lotID'))

In [26]:
gdf.shape

(2496, 25)

In [23]:
gdf.drop(columns=['lots', 'lot', 'cnt_code'], inplace = True)

In [25]:
gdf.drop_duplicates(inplace = True)

In [41]:
len(gdf.wdID.unique())

991

In [29]:
len(gdf.record_ID.unique())

381

In [31]:
set1gdf = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer='WD_Set001_Combined')

In [32]:
set1gdf.columns

Index(['Record_ID', 'wetdet_delin_number', 'trsqq', 'parcel_id',
       'address_location_desc', 'city', 'county', 'site_name', 'site_desc',
       'latitude', 'longitude', 'Coord_Source', 'DocumentName', 'DecisionLink',
       'is_batch_file', 'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'lot_num', 'lot',
       'Match_Year', 'ORTAXLOT', 'Match_found', 'Manual_note', 'Shape_Length',
       'Shape_Area', 'Edits_Complete', 'matcher2', 'geometry'],
      dtype='object')

In [34]:
set1gdf.shape

(2092, 32)

In [33]:
len(set1gdf.Record_ID.unique())

1362

In [44]:
set1gdf[~set1gdf.wetdet_delin_number.isin(gdf.wdID.unique())].wetdet_delin_number.unique()

array(['WD2019-0455', 'WD2020-0302', 'WD2020-0374', 'WD2020-0536',
       'WD2020-0391', 'WD2020-0669', 'WD2021-0162', 'WD2021-0703'],
      dtype=object)

In [46]:
set1gdf[~set1gdf.wetdet_delin_number.isin(gdf.wdID.unique())].Manual_note.unique()

array(['ROW', 'ROW & partial', 'partial', None], dtype=object)